In [43]:
import requests
import json
from bs4 import BeautifulSoup

def get_page(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    json_script = soup.find("script", type="application/ld+json")

    # If we got the content
    if json_script is not None:
        script_content = json.loads(json_script.text)
        # If the last thing in the navigation is "Cocktail Recipes"
        if ("Cocktail Recipes" == script_content[0]['itemListElement'][-1]['item']['name']):
            script_content = json.loads(json_script.text)
            drink_name = script_content[1]['name']
            ingredients = script_content[1]["recipeIngredient"]
            instr = script_content[1]["recipeInstructions"]

            step_list = []
            for step in instr:
                step_list.append(step["text"])

            with open("martha_stewart.json", 'a') as outfile:
                data = {"name": drink_name, "ingredients": ingredients, "instructions": step_list}


In [46]:
get_page("https://www.marthastewart.com/1165530/french-martini")

French Martini
['1 1/2 ounces vodka', '1/2 ounce Chambord', '3/4 ounces pineapple juice', 'Lemon twist, for garnish']
Fill a cocktail shaker halfway with ice. Add in vodka, Chambord, and pineapple juice. Shake well. Strain into a 4-ounce martini glass and garnish with lemon twist.


In [42]:
req = requests.get("https://www.marthastewart.com/314342/extra-dirty-martini")
soup = BeautifulSoup(req.text, "lxml")
json_script = soup.find("script", type="application/ld+json")
script_content = json.loads(json_script.text)
print("Cocktail Recipes" == script_content[0]['itemListElement'][-1]['item']['name'])

True


In [ ]:
# Esquire has somewhat semantic markup for recipes
# Ingredients are in a ingredients-body
# Instructions are in a directions-body
# Can probably filter URLs by having "drinks" in them, like
# <link href="https://www.esquire.com/food-drink/drinks/recipes/a3683/negroni-drink-recipe/" rel="canonical">

    

In [ ]:
# https://www.thecocktaildb.com/api.php
# Probably has some sort of rate limiting or something.
# May end up being an excuse to write a proxy spinner


In [ ]:
#https://makemeacocktail.com/recipes/Gin-cocktails/?ing=61
#https://kindredcocktails.com/cocktail/bananarac
#https://www.thecocktailproject.com/search-recipes
# URLS of the form https://www.diffordsguide.com/cocktails/recipe
#https://www.cocktailcontessa.com
#https://www.allrecipes.com/recipes/133/drinks/cocktails/
#https://www.seriouseats.com/drink-recipes-5117861
#https://imbibemagazine.com/category/recipes/cocktails-spirits-recipes/
#https://mrbostondrinks.com/recipes/blue-lagoon

In [52]:
url = "https://www.pinterest.com/pin/create/link/?url=https://www.marthastewart.com/2126005/halloween-cake-recipes?slide=094fbdda-1f06-42ae-8722-fe180a2413fd%3Futm_source=pinterest.com%26utm_medium=social%26utm_campaign=social-share-gallery%26utm_content=20210921&media=https%3A%2F%2Fimagesvc.meredithcorp.io%2Fv3%2Fmm%2Fimage%3Furl%3Dhttps%253A%252F%252Fassets.marthastewart.com%252Fstyles%252Fwmax-750%252Fd33%252Fcrawly-cake-phobias-1011mld107647%252Fcrawly-cake-phobias-1011mld107647_vert_0_horiz.jpg%253Fitok%253Df2RnN138&description=Creepy-Crawly%20Cake%20"
import re
matcher = re.compile("(http|https)://www\.marthastewart\.com")
print(re.match(matcher, url))
url = "https://www.marthastewart.com/2138473/pastry-chef-dominique-ansel-shares-lessons-learned-martha-stewart"
print(re.match(matcher, url))


None
<_sre.SRE_Match object; span=(0, 29), match='https://www.marthastewart.com'>


In [2]:
import requests
from bs4 import BeautifulSoup
req = requests.get("https://mrbostondrinks.com/recipes/blue-lagoon-2000")
soup = BeautifulSoup(req.text, "lxml")


In [3]:
req.text

'<!doctype html>\n<html class="no-js" ng-app="mrBostonRecipes" lang="en" ng-strict-di>\n  <head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\n    <title ng-bind-template="{{viewTitle}} | Mr. Boston Drinks"></title>\n    <base href="/index.html">\n    <meta property="og:site_name" content="Mr. Boston Drinks"/>\n    <meta name="fragment" content="!">\n    <meta property="fb:app_id" content="948323305227233" />\n    <meta name="p:domain_verify" content="1d1fa1f440b02fc553b3079876039d7a" />\n\n    <link rel="icon" type="image/png" href="/app/img/favicon.png"/>\n    <link rel="apple-touch-icon" href="/app/img/ipad@1x.png" sizes="76x76"/>\n    <link rel="apple-touch-icon" href="/app/img/iphone.png" sizes="120x120"/>\n    <link rel="apple-touch-icon" href="/app/img/ipad@2x.png" sizes="152x152"/>\n    <link rel="apple-touch-icon" href="/app/img/iphone6plus.png" sizes="180x180"/>\n\n\n    \n    \n    <link rel="styleshe

In [3]:
from bs4 import BeautifulSoup
with open("./data/index.html", 'r') as infile:
    html = infile.read()
    bs = BeautifulSoup(html)
    print(bs.find("h1", attrs={"class": "recipe-name"}).text)

Four in Hand


/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
